In [1]:
from build_Case1 import build_Case1
import numpy as np
from scipy.interpolate import interp1d
import pickle
import pandas as pd
import uuid
import time
from pathlib import Path

In [2]:
rootPath = '/nobackup/jakravit/data/'
batch_name = 'test3'
clearIOPprofiles = True

if clearIOPprofiles:
    # clear iop profiles
    %rm -rf /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    %mkdir /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    # clear batch files
    %rm -rf /nobackup/jakravit/data/HE60/run/batch
    %mkdir /nobackup/jakravit/data/HE60/run/batch
    # clear runlist
    %rm /nobackup/jakravit/data/HE60/run/runlist.txt  
    Path('/nobackup/jakravit/data/HE60/run/runlist.txt').touch()
    # clear outputs
    %rm -rf /nobackup/jakravit/data/HE60/output/EcoLight/excel
    %mkdir /nobackup/jakravit/data/HE60/output/EcoLight/excel   

In [3]:
# Data paths 

# phytoplankton SIOP spectral library
phyPath = rootPath+'EAP_phytoplankton_dataset/PFTs.p'
with open(phyPath, 'rb') as fp:
    phy_library = pickle.load(fp)

# NAP spectral libraries
minpath = rootPath+'EAP_NAP_dataset/minerals_V2.p'
with open(minpath, 'rb') as fp:
    datamin = pickle.load(fp)  

detpath = rootPath+'EAP_NAP_dataset/det_V1.p'
with open(detpath, 'rb') as fp:
    datadet = pickle.load(fp) 

# Benthic library
benthic_lib = pd.read_csv(rootPath+'benthic_spec_libary_FT_V4.csv')

# adjacency library
adj_lib = pd.read_csv(rootPath+'adjacency_spectra_V2.csv')

# aeronet library
aero_lib = pd.read_csv(rootPath+'aeronet_invdata_match.csv')

In [4]:
rows = []
uidList = []
for k in range(5):
    iops, cols, row = build_Case1(phy_library, datamin, datadet, benthic_lib, adj_lib, aero_lib)
    rows.append(row)

    #% TOTAL IOPS

    # depth
    depth = iops['Depth']['Depth']
    depth = depth.reshape(-1, 1)
    xfactor = iops['Depth']['xfactor']
    dlen = len(depth)

    # chl profile
    chlProfile = [iops['Phyto']['TotChl'] * x for x in xfactor]
    chlProfile = np.asarray(chlProfile).reshape(-1, 1)
    foot = np.hstack((-1,0))
    chlzProfile = np.hstack((depth,chlProfile))
    chlzProfile = np.vstack((chlzProfile,foot))

    # absorption
    atotphy = iops['Phyto']['a_tot']
    atotmin = iops['Min']['a_tot']
    atotdet = iops['Det']['a_tot']
    atotdom = iops['CDOM']['a_tot'][63:]
    atot = atotphy + atotmin + atotdet + atotdom
    a_tot = [atot * x for x in xfactor]
    a_totphy = [atotphy * x for x in xfactor]
    a_totmin = [atotmin * x for x in xfactor]
    a_totdet = [atotdet * x for x in xfactor]
    a_totdom = [atotdom * x for x in xfactor]

    # Scatter
    btotphy = iops['Phyto']['b_tot']
    btotmin = iops['Min']['b_tot']
    btotdet = iops['Det']['b_tot']
    btot = btotphy + btotmin + btotdet

    # attenuation
    ctotphy = atotphy + btotphy
    ctotmin = atotmin + btotmin
    ctotdet = atotdet + btotdet
    ctotdom = atotdom # no scattering
    ctot = atot + btot
    c_tot = [ctot * x for x in xfactor]
    c_totphy = [ctotphy * x for x in xfactor]
    c_totmin = [ctotmin * x for x in xfactor]
    c_totdet = [ctotdet * x for x in xfactor]
    c_totdom = [ctotdom * x for x in xfactor]

    # backscattering
    bbtotphy = iops['Phyto']['bb_tot']
    bbtotmin = iops['Min']['bb_tot']
    bbtotdet = iops['Det']['bb_tot']
    bbtot = bbtotphy + bbtotmin + bbtotdet
    bb_tot = [bbtot * x for x in xfactor]
    bb_totphy = [bbtotphy * x for x in xfactor]
    bb_totmin = [bbtotmin * x for x in xfactor]
    bb_totdet = [bbtotdet * x for x in xfactor]
    
    #% PREPARE FOR WRITING TO .TXT FILE IN HYDROLIGHT FORMAT

    # lambda
    l = np.arange(400,902.5,2.5)

    llen = len(l)
    line11 = np.hstack((llen,l)).reshape(1,-1)
    # AC data
    foot = np.hstack((-1,[0]*(llen*2)))
    # only phy
    acProfile_phy = np.hstack((depth,a_totphy,c_totphy))
    acProfile_phy = np.vstack((acProfile_phy,foot))
    # only min
    acProfile_min = np.hstack((depth,a_totmin,c_totmin))
    acProfile_min = np.vstack((acProfile_min,foot))
    # only det
    acProfile_det = np.hstack((depth,a_totdet,c_totdet))
    acProfile_det = np.vstack((acProfile_det,foot))
    # only dom
    acProfile_dom = np.hstack((depth,a_totdom,c_totdom))
    acProfile_dom = np.vstack((acProfile_dom,foot))
    # all components
    acProfile = np.hstack((depth,a_tot,c_tot))
    acProfile = np.vstack((acProfile,foot))  

    # bb data
    foot = np.hstack((-1,[0]*(l)))
    # only phy
    bbProfile_phy = np.hstack((depth,bb_totphy))
    bbProfile_phy = np.vstack((bbProfile_phy,foot))
    # only min
    bbProfile_min = np.hstack((depth,bb_totmin))
    bbProfile_min = np.vstack((bbProfile_min,foot))
    # only det
    bbProfile_det = np.hstack((depth,bb_totdet))
    bbProfile_det = np.vstack((bbProfile_det,foot))
    # all components
    bbProfile = np.hstack((depth,bb_tot))
    bbProfile = np.vstack((bbProfile,foot))

    #% WRITE TO TXT
    
    import string
    import random
    import os 

    iopPath = rootPath+'HE60/iop_profiles/{}/'.format(batch_name)
    if not os.path.exists(iopPath):
        os.mkdir(iopPath)
        
    def random_uid():
        alphabet = string.ascii_lowercase + string.digits
        return ''.join(random.choices(alphabet, k=10))
    uid = random_uid()
    uidList.append(uid)

    # sname.append(uid)
    iopzpath = iopPath+'{}/'.format(uid)
    if not os.path.exists(iopzpath):
        os.mkdir(iopzpath)
    # os.mkdir(rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid))
    # iopzpath = rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid)

    # acData

    # fnames
    fnames = {'total': {'acfname': 'acDatatot_' + uid + '.txt',
                        'bbfname': 'bbDatatot_' + uid + '.txt',
                        'acdata': acProfile,
                        'bbdata': bbProfile},
              'phyto': {'acfname': 'acDataphy_' + uid + '.txt',
                        'bbfname': 'bbDataphy_' + uid + '.txt',
                        'acdata': acProfile_phy,
                        'bbdata': bbProfile_phy},
              'min':   {'acfname': 'acDatamin_' + uid + '.txt',
                        'bbfname': 'bbDatamin_' + uid + '.txt',
                        'acdata': acProfile_min,
                       'bbdata': bbProfile_min},
              'det':   {'acfname': 'acDatadet_' + uid + '.txt',
                        'bbfname': 'bbDatadet_' + uid + '.txt',
                        'acdata': acProfile_det,
                        'bbdata': bbProfile_det},
              }

    # write chl data
    header = '\n'.join(['CHL CONC DATA PROFILE',
                        'SWIPE HE data files',
                        '#',
                        '#','#','#','#','#','#','#\n'])
    with open(os.path.join(iopzpath,'chlzData_{}.txt'.format(uid)),'w') as f:
        f.writelines(header)
        np.savetxt(f,chlzProfile,delimiter='\t')

    # write AC data
    header = '\n'.join(['TOTAL AC DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['acfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11,fmt='%d',delimiter='\t')
            np.savetxt(f,fnames[fname]['acdata'],delimiter='\t')

    # write bb data
    header = '\n'.join(['TOTAL BB DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['bbfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11,fmt='%d',delimiter='\t')
            np.savetxt(f,fnames[fname]['bbdata'],delimiter='\t')

    # write iroot file        
    from write_iroot import iroot
    iroot(l, iops, batch_name, uid, rootPath)
            
sims = pd.DataFrame(rows,columns=cols, index=uidList)
sims.to_csv(rootPath + 'HE60/batch_data/{}_table.csv'.format(batch_name))
print ('DONE!')

DONE!


In [5]:
sims.shape

(5, 10670)

In [6]:
sims.head()

,TotChl,ph_a_tot_400.0,ph_a_tot_402.5,ph_a_tot_405.0,ph_a_tot_407.5,ph_a_tot_410.0,ph_a_tot_412.5,ph_a_tot_415.0,ph_a_tot_417.5,ph_a_tot_420.0,...,atm_Std-F,atm_VolC-C,atm_REff-C,atm_VMR-C,atm_Std-C,prof,OZA,OAA,SZA,SAA
v6isaehcpf,0.032,0.000794,0.000806,0.000812,0.000823,0.000831,0.000839,0.000844,0.000848,0.000849,...,0.491,0.299,1.939,2.203,0.512,afglss,30,105,55,40
s1w6f4ys97,0.147,0.005542,0.005673,0.005776,0.005901,0.006008,0.006103,0.006166,0.006212,0.006232,...,0.398,0.109,2.359,2.817,0.583,afglmw,25,60,25,50
gfw996fpws,0.207,0.002641,0.002707,0.002731,0.002764,0.002785,0.002807,0.002820,0.002831,0.002836,...,0.364,0.122,2.159,2.774,0.680,afglss,25,90,60,60
kmdx0971hc,0.259,0.011520,0.011766,0.012065,0.012416,0.012772,0.013129,0.013439,0.013687,0.013828,...,0.419,0.120,2.200,2.812,0.674,afglt,20,70,45,60
es6v0f7r65,0.178,0.006112,0.006114,0.006119,0.006128,0.006135,0.006134,0.006127,0.006115,0.006095,...,0.486,0.054,3.272,3.902,0.581,afglt,25,110,40,60
